In [30]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# Charger les données
df = pd.read_csv("dataVisualization/result.csv")

# Supprimer les valeurs manquantes
df.dropna(inplace=True)

# Supprimer les valeurs 'Bee & Bumblebee'
df = df[df['bug type'] != 'Bee & Bumblebee']

# Sélectionner les colonnes de caractéristiques et la cible
train_predictor_columns = df.columns.difference(['bug type', 'species'])
target_labels = df['bug type']
train_feats = df[train_predictor_columns]

# Encodage des étiquettes de la cible
y_encoded, y_labels = pd.factorize(target_labels)

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(train_feats, y_encoded, test_size=0.2, random_state=42)

# Normalisation des données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Définir les paramètres pour MLP et SVM
mlp = MLPClassifier(hidden_layer_sizes=(32, 16, 8), alpha=0.001, max_iter=2000, solver='adam', random_state=0, tol=1e-9)
svm = SVC(C=100, kernel='linear', degree=3, gamma='scale', coef0=0, probability=True)

# Entraîner les modèles individuellement
mlp.fit(X_train_scaled, y_train)
svm.fit(X_train_scaled, y_train)

# Obtenir les prédictions de chaque modèle
y_prob_mlp = mlp.predict_proba(X_test_scaled)
y_prob_svm = svm.predict_proba(X_test_scaled)

# Obtenir les classes
classes = mlp.classes_

# Fusionner les résultats des deux modèles en utilisant des règles conditionnelles strictes
final_pred = []
bee_index = np.where(classes == y_labels.tolist().index('Bee'))[0][0]
bumblebee_index = np.where(classes == y_labels.tolist().index('Bumblebee'))[0][0]

for i in range(len(y_test)):
    if y_prob_svm[i][bumblebee_index] > 0.8:  # Seuil strict pour Bumblebee
        final_pred.append(bumblebee_index)
    elif y_prob_mlp[i][bee_index] > 0.7:  # Seuil strict pour Bee
        final_pred.append(bee_index)
    else:
        # Si aucune condition n'est satisfaite, utiliser la classe avec la plus haute probabilité globale
        final_pred.append(np.argmax(y_prob_mlp[i]) if y_prob_mlp[i].max() > y_prob_svm[i].max() else np.argmax(y_prob_svm[i]))

# Correction de l'erreur en spécifiant explicitement les étiquettes des classes
unique_labels = np.unique(y_test)
classification_report = metrics.classification_report(y_test, final_pred, labels=unique_labels, target_names=y_labels[unique_labels])
print("Classification Report:")
print(classification_report)

# Export des résultats fusionnés
test_df = df.loc[X_test.index]
test_df.drop(train_predictor_columns, axis=1, inplace=True)
test_df.drop(['species'], axis=1, inplace=True)
test_df['Fusion_Prediction'] = y_labels[final_pred]
test_df['Fusion_Recognition'] = test_df['bug type'] == test_df['Fusion_Prediction']
test_df.to_csv("fusion_results.csv", index=False)


c:\Users\leona\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


Classification Report:
              precision    recall  f1-score   support

         Bee       0.91      0.70      0.79        30
   Butterfly       0.00      0.00      0.00         5
        Wasp       1.00      0.50      0.67         2
   Bumblebee       0.48      0.92      0.63        13

   micro avg       0.69      0.68      0.69        50
   macro avg       0.60      0.53      0.52        50
weighted avg       0.71      0.68      0.67        50



c:\Users\leona\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\leona\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\leona\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif